In [6]:
using Revise
using Bilevel

using RigidBodyDynamics
using ForwardDiff

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)

floor = findbody(mechanism, "floor")
point = Point3D(default_frame(floor), [0.,0.,0.])
normal = FreeVector3D(default_frame(floor), [0.,0.,1.])
floor_obs = Obstacle(floor, point, normal, :xyz, .5)

obstacles = [floor_obs]

1-element Array{Obstacle,1}:
 Obstacle(RigidBody: "floor", Point3D in "after_floor_to_world": [0.0, 0.0, 0.0], FreeVector3D in "after_floor_to_world": [0.0, 0.0, 1.0], [1.0 6.12323e-17 -1.0 -1.83697e-16; 0.0 1.0 1.22465e-16 -1.0; 0.0 0.0 0.0 0.0], 0.5, false)

In [3]:
env = Environment(mechanism, urdf, obstacles)

Environment(Contact[Contact(Spanning tree:
Vertex: world (root)
  Vertex: floor, Edge: floor_to_world
    Vertex: ball, Edge: floor_to_ball
No non-tree joints., ball, Point3D in "after_floor_to_ball": [0.0, 0.0, 0.0], Obstacle(floor, Point3D in "after_floor_to_world": [0.0, 0.0, 0.0], FreeVector3D in "after_floor_to_world": [0.0, 0.0, 1.0], [1.0 6.12323e-17 -1.0 -1.83697e-16; 0.0 1.0 1.22465e-16 -1.0; 0.0 0.0 0.0 0.0], 0.5, false))])

In [4]:
sim_data = get_sim_data_indirect(mechanism,env,.01,relax_comp=false)

SimData(Spanning tree:
Vertex: world (root)
  Vertex: floor, Edge: floor_to_world
    Vertex: ball, Edge: floor_to_ball
No non-tree joints., Environment(Contact[Contact(Spanning tree:
Vertex: world (root)
  Vertex: floor, Edge: floor_to_world
    Vertex: ball, Edge: floor_to_ball
No non-tree joints., ball, Point3D in "after_floor_to_ball": [0.0, 0.0, 0.0], Obstacle(floor, Point3D in "after_floor_to_world": [0.0, 0.0, 0.0], FreeVector3D in "after_floor_to_world": [0.0, 0.0, 1.0], [1.0 6.12323e-17 -1.0 -1.83697e-16; 0.0 1.0 1.22465e-16 -1.0; 0.0 0.0 0.0 0.0], 0.5, false))]), StateCache{…}(…), StateCache{…}(…), EnvironmentJacobianCache{…}(…), 0.01, Bilevel.VariableSelector(Dict(:qnext=>1:7,:vnext=>8:13,:β1=>14:17,:λ1=>18:18,:c_n1=>19:19), 19), Bilevel.ConstraintSelector(Dict(:dyn=>8:13,:fric_β_comp1=>15:18,:ϕ_c_n_comp1=>14:14,:kin=>1:7,:cone_λ_comp1=>19:19), 19, Dict(:ϕ_pos1=>7:7,:fric_pos1=>8:11,:c_n_pos1=>6:6,:β_pos1=>1:4,:λ_pos1=>5:5,:cone_pos1=>12:12), 12), :generate_solver_fn_sim_ind

In [18]:
sim_data.envj_cache[ForwardDiff.Dual]

EnvironmentJacobian{ForwardDiff.Dual}(Bilevel.ContactJacobian{ForwardDiff.Dual}[ContactJacobian{Dual}(Contact(Spanning tree:
Vertex: world (root)
  Vertex: floor, Edge: floor_to_world
    Vertex: ball, Edge: floor_to_ball
No non-tree joints., ball, Point3D in "after_floor_to_ball": [0.0, 0.0, 0.0], Obstacle(floor, Point3D in "after_floor_to_world": [0.0, 0.0, 0.0], FreeVector3D in "after_floor_to_world": [0.0, 0.0, 1.0], [1.0 6.12323e-17 -1.0 -1.83697e-16; 0.0 1.0 1.22465e-16 -1.0; 0.0 0.0 0.0 0.0], 0.5, false)), [0.0 9.81 … -9.81 -1.80207e-15; 0.0 0.0 … 1.20138e-15 -9.81; 9.81 0.0 … 0.0 0.0], Dual{Nothing}(0.0), ForwardDiff.Dual[Dual{Nothing}(1.0) Dual{Nothing}(0.0) Dual{Nothing}(0.0); Dual{Nothing}(0.0) Dual{Nothing}(1.0) Dual{Nothing}(0.0); Dual{Nothing}(0.0) Dual{Nothing}(0.0) Dual{Nothing}(1.0)], ForwardDiff.Dual[Dual{Nothing}(0.0), Dual{Nothing}(0.0), Dual{Nothing}(0.0)], GeometricJacobian: body: "world", base: "after_floor_to_ball", expressed in "world":
ForwardDiff.Dual[Dual{No

In [20]:
sim_data.vs.num_vars

19

In [22]:
q0 = [1., 0., 0., 0., 0., 0., 0.]
v0 = [0., 0., 0., 0., 0., 0.]
u0 = [0., 0., 0., 0., 0., 0.]
dyn = generate_solver_fn_sim_indirect(sim_data, q0, v0, u0)
dyn(vcat(q0,v0,zeros(6)))

31-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [38]:
ForwardDiff.jacobian(dyn,vcat(q0,v0,zeros(6)))

31×19 Array{Float64,2}:
   2.27558e-314  NaN             …  NaN             NaN           
 NaN               2.27558e-314       1.9098e-312     1.84614e-312
   2.27558e-314    2.27558e-314       3.37397e-312    3.33153e-312
 NaN             NaN                  4.71083e-312    4.56229e-312
   2.27558e-314    2.27558e-314       2.54639e-313    2.75859e-313
 NaN             NaN             …    6.28111e-312    6.13257e-312
   2.27558e-314  NaN                  2.97079e-313    2.97079e-313
   2.27558e-314    2.27558e-314       6.98137e-312    6.89649e-312
   2.27558e-314  NaN                  3.60739e-313    3.60739e-313
 NaN               2.27558e-314     NaN             NaN           
   2.27558e-314    2.34903e-314  …    1.0228e-311     1.00158e-311
 NaN               2.34791e-314     NaN             NaN           
   2.27558e-314  NaN                  4.45619e-313    4.45619e-313
   ⋮                             ⋱                                
 NaN               2.27558e-314       